In [ ]:
# imports
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np

In [ ]:
driver = webdriver.Chrome("/Applications/chromedriver")
url = "https://neos-guide.org/content/diet-problem-solver"
driver.get(url)
content = driver.page_source
soup = BeautifulSoup(content)

In [ ]:
foods = pd.DataFrame()
table = soup.find('div', attrs={'id':'foods_selection'}).find('table').find('tbody')

headers = []
for header in table.findAll('tr', recursive=False)[1].findAll('td', recursive=False)[1:]:
    headers.append(header.findAll(text=True)[0])
    
for row in table.findAll('tr', recursive=False)[2:]:
    entries = []
    for cell in row.findAll('td', recursive=False)[1:]:
        if cell.find('input') is not None:
            entries.append(str(cell.find('input')).split('=')[-1].split('"')[1])
        else:
            entries.append((cell.findAll(text=True)[0]))
    foods = foods.append({headers[i] : entries[i] for i in range(5)}, ignore_index=True)
    
foods = foods[headers]

In [ ]:
nutrients = pd.DataFrame()
table = soup.find('div', attrs={'id':'nutrients_selection'}).find('table').find('tbody')

headers = []
for header in table.findAll('tr', recursive=False)[1].findAll('td', recursive=False)[1:]:
    headers.append(header.findAll(text=True)[0])
    
for row in table.findAll('tr', recursive=False)[2:]:
    entries = []
    for cell in row.findAll('td', recursive=False)[1:]:
        if cell.find('input') is not None:
            entries.append(str(cell.find('input')).split('=')[-1].split('"')[1])
        else:
            entries.append((cell.findAll(text=True)[0]))
    nutrients = nutrients.append({headers[i] : entries[i] for i in range(4)}, ignore_index=True)
    
nutrients = nutrients[headers]

In [ ]:
nutrients_info = pd.DataFrame()
table = soup.find('div', attrs={'id':'nutrient_info'}).find('table').find('tbody')
table
headers = []
for header in table.findAll('tr', recursive=False)[0].findAll('td', recursive=False):
    headers.append(header.findAll(text=True)[0])
    
for row in table.findAll('tr', recursive=False)[1:]:
    entries = []
    for cell in row.findAll('td', recursive=False):
        if len(cell.findAll(text=True)) != 0:
            entries.append((cell.findAll(text=True)[0]))
    if len(entries) == 12:
        nutrients_info = nutrients_info.append({headers[i] : entries[i] for i in range(12)}, ignore_index=True)
        
nutrients_info = nutrients_info[headers]

In [ ]:
nutrients.head()

In [ ]:
foods = foods.merge(nutrients_info, on='Name', how='left').dropna()
foods.head()

In [ ]:
# export
foods.set_index('Name').rename(columns={'Price/Serving ($)' : 'Cost'}).to_csv('neos_foods.csv')
nutrients.set_index('Name').to_csv('neos_nutrients.csv')